### This document trains a RF on metrics and a GRU on hydrographs.

In [1]:
%load_ext autoreload
%autoreload 2
import copy
import torch
import itertools
from collections import defaultdict

from torch import nn
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import xarray
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from tqdm import tqdm
from pathlib import Path
from neuralhydrology.evaluation.metrics import calculate_all_metrics

df = pd.read_csv('data/rmh-stage1.csv', index_col=0)

/system/apps/mlsoft/conda/group-bioinf/gauch/envs/cf/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Load simulations and observations

In [2]:
def load_data(base_dir):
    obs_file = base_dir / 'all_gauges.nc'
    if not obs_file.exists():
        raise ValueError(f'Observations netCDF file not found at {obs_file}')
    # netcdfs only have a numeric dimension "nstations" that maps to the station_id variable.
    # for easier processing, we directly make the station_id the dimension.
    obs = xarray.load_dataset(obs_file).swap_dims({'nstations': 'station_id'})

    # load hydrographs from individual models
    hydrographs = {'Q': obs['Q']}
    model_dirs = [d for d in base_dir.glob('model/*') if d.is_dir()]
    for model_dir in model_dirs:
        model_name = model_dir.name
        model_nc = list(model_dir.glob('*.nc'))
        if len(model_nc) != 1:
            if len(model_nc) == 0:
                continue
        hydrographs[model_name] = xarray.open_dataset(model_nc[0]).swap_dims({'nstations': 'station_id'})['Q']

    hydrograph_xr = xarray.concat(hydrographs.values(), dim='model')
    hydrograph_xr['model'] = list(hydrographs.keys())
    return hydrograph_xr

OBJECTIVES = ['objective_1/great-lakes/validation-temporal', 'objective_2/great-lakes/validation-temporal']
XR = {obj: load_data(Path(f'data/{obj.split("/")[0]}')) for obj in OBJECTIVES}

Calculate metrics for each model

In [3]:
metric_vals = defaultdict(dict)
for obj, model in itertools.product(OBJECTIVES, df['model_a'].unique()):
    sub_df = df[(df['objective']==obj) & ((df['model_a']==model) | (df['model_b']==model))]
    basins_dates = sub_df[['basin', 'start_date', 'end_date']].drop_duplicates()
    for _, (basin, start_date, end_date) in basins_dates.iterrows():
        sub_xr = XR[obj].sel(station_id=basin, time=slice(start_date, end_date))
        setting_metrics = calculate_all_metrics(sub_xr.sel(model='Q'), sub_xr.sel(model=model), datetime_coord='time')
        for metric, val in setting_metrics.items():
            metric_vals[metric][(obj, model, basin, start_date)] = val

metric_vals = pd.DataFrame(metric_vals)
metric_vals.index = metric_vals.index.set_names(('objective', 'model', 'basin', 'start_date'))

Prepare input data for the GRU and the RF

In [4]:
def prepare_data(df, targets):
    metric_df = df.copy()
    input_cols = []
    # Prepare columns that will contain the metrics in metric_df
    for ab in ['a', 'b']:
        for metric in metric_vals.columns:
            input_cols.append(f'model_{ab}_{metric}')
            metric_df[input_cols[-1]] = np.nan

    task_onehot = torch.from_numpy(pd.get_dummies(df['task']).values)
    timeseries = []
    y = []
    dropped_idx = []
    for i, (idx, rating) in enumerate(df.iterrows()):
        obj = metric_df.loc[idx, 'objective']
        basin = metric_df.loc[idx, 'basin']
        start_date = metric_df.loc[idx, 'start_date']

        # GRU inputs: observations & simulations from model a and b & task one-hot encoding
        basin_xr = XR[obj].sel(time=slice(start_date, rating['end_date']), station_id=basin)
        model_a_ts = basin_xr.sel(model=rating['model_a']).values
        model_b_ts = basin_xr.sel(model=rating['model_b']).values
        qobs = basin_xr.sel(model='Q').values
        if any(np.any(np.isnan(ts)) for ts in [model_a_ts, model_b_ts, qobs]):
            dropped_idx.append(idx)
            continue
        
        # RF inputs: metrics from model a and b & task one-hot encoding
        for metric in metric_vals.columns:
            for ab in ['a', 'b']:
                model = metric_df.loc[idx, f'model_{ab}']
                metric_df.loc[idx, f'model_{ab}_{metric}'] = metric_vals.loc[(obj, model, basin, start_date), metric]

        # some timeseries have 731 entries. for simplicity, we ignore that last entry where it exists.
        ts_inputs = torch.stack([torch.from_numpy(x) for x in [model_a_ts, model_b_ts, qobs]], dim=1)[:730]
        ts_inputs = torch.cat([ts_inputs, task_onehot[i].unsqueeze(0).repeat(ts_inputs.shape[0], 1)], dim=1)
        timeseries.append(ts_inputs)

        # Target: rating encoded as integer
        y.append(np.argmax(rating[targets].astype(int).values))

    if len(dropped_idx) > 0:
        metric_df = metric_df.loc[np.setdiff1d(metric_df.index, dropped_idx)]
        print(f'Dropped {len(dropped_idx)} NaN samples, remaining {len(metric_df)}.')

    # Convert to tensors that can be used with PyTorch
    x_gru = torch.stack(timeseries)
    # Add task encoding to the metrics df that will be the inputs to the RF
    x_rf = pd.concat([metric_df[input_cols], pd.get_dummies(metric_df['task'])], axis=1).reset_index(drop=True)
    y = torch.tensor(y).to(torch.long)

    return x_rf, x_gru, y

Prepare train/val/test splits for GRU and RF

In [5]:
def get_data(df, target_names):

    # Get input and target data
    x_rf, x_gru, y = prepare_data(df, target_names)
    
    targets = target_names
    np.random.seed(0)
    torch.manual_seed(0)

    x_sub_rf = x_rf
    x_sub_gru = x_gru
    y_sub = y

    # 5-fold CV
    shuffled_indices = np.random.permutation(x_sub_gru.shape[0])
    kf = KFold(n_splits=5, shuffle=False)
    idx_train, idx_val, idx_test = [], [], []
    for i, (idx_a, idx_b) in enumerate(kf.split(x_sub_rf.iloc[shuffled_indices].values)):
        # Further split the train fold into train and val
        n_train = int(idx_a.shape[0] * 0.8)
        idx_train.append(idx_a[:n_train])
        idx_val.append(idx_a[n_train:])
        idx_test.append(idx_b)

    return x_sub_rf, x_sub_gru, y_sub, idx_train, idx_val, idx_test, targets

In [6]:
def evaluate(y_sub, y_hat, targets):
    report = classification_report(y_sub, y_hat, target_names=targets, output_dict=True, zero_division=0)
    confusion = confusion_matrix(y_sub, y_hat, normalize='pred', labels=list(range(len(targets))))

    return report, confusion

def display_eval(reports, confusions, targets):
    print(f'Average accuracy: {np.mean([report["accuracy"] for report in reports]):.3f}')
    display(sum(pd.DataFrame({t: report[t] for t in targets}) for report in reports) / len(reports))

    display((sum(pd.DataFrame(confusion,
                            index=[f'true {t}' for t in targets],
                            columns=[f'predicted {t}' for t in targets])
                 for confusion in confusions) / len(confusions)).style.background_gradient(cmap='Greens', axis=0))

In [7]:
def run_rf(x_sub, y_sub, idx_train, idx_val, idx_test, targets, test=False):

    reports, confusions = [], []
    val_indices = idx_val if not test else idx_test
    for i, (i_train, i_val) in enumerate(zip(idx_train, val_indices)):
        if i == 0:
            print(f'{len(i_train)} training samples, {len(i_val)} validation/test samples, {x_sub.shape[1]} features.')

        model = RandomForestClassifier(n_estimators=1000, max_depth=10, random_state=0, n_jobs=-1)
        model.fit(x_sub.iloc[i_train], y_sub[i_train])

        y_hat_val = model.predict(x_sub.iloc[i_val])

        report, confusion = evaluate(y_sub[i_val], y_hat_val, targets)
        reports.append(report)
        confusions.append(confusion)
    display_eval(reports, confusions, targets=targets)

    return model, reports, confusions

In [8]:
def run_gru(x_sub, y_sub, idx_train, idx_val, idx_test, targets, hidden_size, lr, bs, dropout, test=False):
    ds = torch.utils.data.TensorDataset(x_sub, y_sub)

    best_val_reports = []
    best_val_confusions = []
    test_reports = []
    test_confusions = []
    for i, (i_train, i_val, i_test) in enumerate(zip(idx_train, idx_val, idx_test)):
        np.random.seed(0)
        torch.manual_seed(0)
    
        # Initialize model and optimizer
        device = 'cuda:0'
        gru = nn.GRU(input_size=x_sub.shape[2], hidden_size=hidden_size, batch_first=True).to(device)
        head = nn.Linear(gru.hidden_size, len(targets)).to(device)
        dropout_layer = nn.Dropout(p=dropout)
        loss_fn = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(params=list(gru.parameters()) + list(head.parameters()), lr=lr)
        
        # Create train/val/test dataloaders
        train_ds = torch.utils.data.Subset(ds, indices=i_train)
        val_ds = torch.utils.data.Subset(ds, indices=i_val)
        test_ds = torch.utils.data.Subset(ds, indices=i_test)
        train_loader = torch.utils.data.DataLoader(train_ds, batch_size=bs, shuffle=True)
        val_loader = torch.utils.data.DataLoader(val_ds, batch_size=bs, shuffle=False)
        test_loader = torch.utils.data.DataLoader(test_ds, batch_size=bs, shuffle=False)

        if i == 0:
            print(f'{len(train_ds)} training samples, {len(val_ds)} validation, {len(test_ds)} test samples.')

        # Run training
        train_losses = {}
        val_reports = {}
        prev_acc = -np.inf
        best_ep = None
        best_model = None
        patience = 20
        for epoch in range(500):
            gru.train(), head.train()
            total_loss = 0.0
            for batch_x, batch_y in train_loader:
                optimizer.zero_grad()
                gru_out, _ = gru(batch_x.to(device))
                head_out = head(dropout_layer(gru_out[:, -1]))
                loss = loss_fn(head_out, batch_y.to(device))
                loss.backward()
                optimizer.step()

                total_loss += loss.detach().cpu().item()
            
            train_losses[epoch] = total_loss / len(train_loader)
            # Run evaluation every few epochs
            if epoch % 5 == 0:
                gru.eval(), head.eval()
                y_val = []
                y_hat = []
                with torch.no_grad():
                    for batch_x, batch_y in val_loader:
                        gru_out, _ = gru(batch_x.to(device))
                        head_out = head(gru_out[:, -1])
                        y_hat.append(head_out.cpu())
                        y_val.append(batch_y)
                y_val = torch.cat(y_val)
                y_hat = torch.cat(y_hat)
                y_hat_cls = y_hat.argmax(dim=1)
                
                report, confusion = evaluate(y_val, y_hat_cls, targets)
                val_reports[epoch] = report, confusion
                print(f'Fold {i} Epoch {str(epoch).ljust(5)} train loss: {train_losses[epoch]:.3f}, validation accuracy: {report["accuracy"]:.3f}')
                # Check for early-stopping based on validation results
                if report['accuracy'] > prev_acc:
                    prev_acc = report['accuracy']
                    best_ep = epoch
                    best_model = copy.deepcopy(gru), copy.deepcopy(head)
                    best_model[0].flatten_parameters()
                if best_ep + patience < epoch:
                    print('early stopping.')
                    break
        
        best_val_reports.append(val_reports[best_ep][0])
        best_val_confusions.append(val_reports[best_ep][1])

        # Run final test set evaluation based on best validation epoch
        if test:
            gru, head = best_model
            gru.eval(), head.eval()
            y_test = []
            y_hat = []
            with torch.no_grad():
                for batch_x, batch_y in test_loader:
                    gru_out, _ = gru(batch_x.to(device))
                    head_out = head(gru_out[:, -1])
                    y_hat.append(head_out.cpu())
                    y_test.append(batch_y)
            y_test = torch.cat(y_test)
            y_hat = torch.cat(y_hat)
            y_hat_cls = y_hat.argmax(dim=1)
            
            report, confusion = evaluate(y_test, y_hat_cls, targets)
            test_reports.append(report)
            test_confusions.append(confusion)

        torch.cuda.empty_cache()
    
    if test:
        display_eval(test_reports, test_confusions, targets=targets)
        return test_reports, test_confusions
    return best_val_reports, best_val_confusions

In [9]:
target_names = ['num_a_wins', 'num_b_wins', 'num_equal_good', 'num_equal_bad']

x_rf, x_gru, y, idx_train, idx_val, idx_test, targets = get_data(df, target_names=target_names)
print(f'shapes  - x_rf: {x_rf.shape}, x_gru: {x_gru.shape}, y: {y.shape}. Targets: {targets}')
print(f'samples - folds: {len(idx_train)}. train: {len(idx_train[0])}, val: {len(idx_val[0])}, test: {len(idx_test[0])}')

Dropped 572 NaN samples, remaining 14014.
shapes  - x_rf: (14014, 27), x_gru: torch.Size([14014, 730, 6]), y: torch.Size([14014]). Targets: ['num_a_wins', 'num_b_wins', 'num_equal_good', 'num_equal_bad']
samples - folds: 5. train: 8968, val: 2243, test: 2803


Run the RF

In [10]:
_ = run_rf(x_rf, y.numpy(), idx_train, idx_val, idx_test, targets=targets, test=True)

8968 training samples, 2803 validation/test samples, 27 features.
Average accuracy: 0.498


,num_a_wins,num_b_wins,num_equal_good,num_equal_bad
precision,0.520748,0.502896,0.315512,0.432573
recall,0.634081,0.749276,0.024043,0.245241
f1-score,0.571341,0.601032,0.044467,0.310973
support,816.400000,956.400000,401.600000,628.400000


,predicted num_a_wins,predicted num_b_wins,predicted num_equal_good,predicted num_equal_bad
true num_a_wins,0.520748,0.142378,0.272523,0.245816
true num_b_wins,0.149660,0.502896,0.273401,0.236002
true num_equal_good,0.138413,0.157487,0.315512,0.085609
true num_equal_bad,0.191179,0.197239,0.138565,0.432573


Run the GRU

In [11]:
h = 265
lr = 5e-4
bs = 1024
dropout = 0.3
reports, confusions = run_gru(x_gru, y, idx_train, idx_val, idx_test, targets=targets, hidden_size=h, lr=lr, bs=bs, dropout=dropout, test=True)

8968 training samples, 2243 validation, 2803 test samples.
Fold 0 Epoch 0     train loss: 1.383, validation accuracy: 0.346
Fold 0 Epoch 5     train loss: 1.330, validation accuracy: 0.381
Fold 0 Epoch 10    train loss: 1.298, validation accuracy: 0.404
Fold 0 Epoch 15    train loss: 1.271, validation accuracy: 0.438
Fold 0 Epoch 20    train loss: 1.226, validation accuracy: 0.463
Fold 0 Epoch 25    train loss: 1.201, validation accuracy: 0.477
Fold 0 Epoch 30    train loss: 1.181, validation accuracy: 0.483
Fold 0 Epoch 35    train loss: 1.172, validation accuracy: 0.495
Fold 0 Epoch 40    train loss: 1.160, validation accuracy: 0.498
Fold 0 Epoch 45    train loss: 1.155, validation accuracy: 0.504
Fold 0 Epoch 50    train loss: 1.135, validation accuracy: 0.503
Fold 0 Epoch 55    train loss: 1.121, validation accuracy: 0.506
Fold 0 Epoch 60    train loss: 1.111, validation accuracy: 0.508
Fold 0 Epoch 65    train loss: 1.094, validation accuracy: 0.504
Fold 0 Epoch 70    train loss: 

,num_a_wins,num_b_wins,num_equal_good,num_equal_bad
precision,0.520403,0.526642,0.335079,0.383861
recall,0.662700,0.718184,0.074181,0.219543
f1-score,0.581030,0.606967,0.115385,0.271769
support,816.400000,956.400000,401.600000,628.400000


,predicted num_a_wins,predicted num_b_wins,predicted num_equal_good,predicted num_equal_bad
true num_a_wins,0.520403,0.127784,0.195308,0.248437
true num_b_wins,0.142867,0.526642,0.298440,0.256073
true num_equal_good,0.133748,0.146834,0.335079,0.111629
true num_equal_bad,0.202982,0.198740,0.171172,0.383861
